### IMU予積分(IMU Pre-integration)
IMU予積分(IMU Pre-integration)はVisual Odometry or SLAMでよく使われた手法です、フレーム間のIMU情報を制約として最適化フレームワークに導入することで、より高精度なOdometry or SLAM
を実現できる。しかし、IMU予積分予積分の式はすごく難しい、どうやって導出しているかどうかについて論文から説明がありません、この文章の目的はIMU予積分の式を導出します。

VINS:midPointIntegration

### IMUのモデル
Gyroモデル
$$\tilde{\omega}_b= \omega_b + bg + ng\tag{1}$$

* $\tilde{\omega}_b$はGyroの測量値, 下付き文字はbはimuのbody座標計を示す
* $\omega_b$はGyroの真値
* $bg$はGyroのbias
* $ng$はGyroのnoise

加速度計モデル
$$\tilde{a}_b= q_{bw}(a_w+g_w) + ba + na\tag{2}$$
* $\tilde{a}_b$は加速度の測量値, 下付き文字はbはimuのbody座標計を示す
* $q_bw$世界座標系をbody座標系に変換するためのクオータニオン
* $a_w$は加速度の真値(世界座標系)
* $g_w$は重量加速度(世界座標系)
* $ba$は加速度のbias
* $na$は加速度のnoise


### 連続時間でIMUの運動モデル
i時刻からj時刻まで
位置(position)：
$$p_{wb_{j}}= p_{wb_{i}} + v^w_i\Delta t - \frac{1}{2} g^w \Delta t^2 + q_{wb_{i}} \int\int_{t\in [i,j]}(q_{b_i b_t}a^{b_t})\delta t^2 \tag{3} $$

速度(velocity)：
$$v_j^w= v_i^w + g^w\Delta t + + q_{wb_{i}} \int_{t\in [i,j]}(q_{b_{i} b_{t}}a^{b_t})\delta t \tag{4} $$

姿勢(quaternion)：
$$q_{wb_{j}}= \int_{t\in [i,j]} q_{wb_{t}} \otimes \left[
\begin{matrix} 0 \\ \omega^{b_t}/2 \end{matrix}
\right] \delta t \tag{5} $$
  
  
(3)(4)(5)で、以下の分量はIMUの測量値のみ関係するため、予積分量と呼びます
$$\alpha_{b_i b_j} =\int\int_{t\in [i,j]}(q_{b_i b_t}a^{b_t})\delta t^2  \tag{6}$$
$$\beta_{b_i b_j} =\int_{t\in [i,j]}(q_{b_{i} b_{t}}a^{b_t})\delta t  \tag{7}$$
$$q_{b_i b_j} =\int_{t\in [i,j]} q_{wb_{t}} \otimes \left[
\begin{matrix} 0 \\ \omega^{b_t}/2 \end{matrix}
\right] \delta t  \tag{8}$$




### IMUの運動モデルの離散化表現
ノイズを考慮せず、imuデータの各フレームの状態変化：

$$p_{wb_{k+1}}= p_{wb_{k}} + v^w_{b_k}\Delta{t} + \frac{1}{2} a \Delta{t^2}  \tag{9} $$
$$q_{w b_{k+1}} = q_{wb_{t}} \otimes \left[
\begin{matrix} 1 \\ \omega \delta t /2 \end{matrix}
\right] \tag{10} $$
$$v^w_{k+1}= v_k^w + a \Delta{t} \tag{11} $$
$$b^{a}_{k+1} = b^{a}_{k} \tag{12} $$
$$b^{g}_{k+1} = b^{g}_{k} \tag{13} $$


here:
 $$ a = \frac{1}{2} [ R_{wb_{k}}(a^{b_k} - b^a_k) - g^w + R_{w b_{k+1}}(a^{b_{k+1}} -b^a_k)-g^w ] $$
 $$  = \frac{1}{2} [ R_{wb_{k}}(a^{b_k} - b^a_k) + R_{w b_{k+1}}(a^{b_{k+1}} -b^a_k) ] - g^w \tag{14} $$
 $$ \omega = \frac{1}{2} [ (\omega^{b_k} - b^g_k) + (\omega^{b_{k+1}} -b^g_k) ]  $$ 
 $$  = \frac{1}{2} (\omega^{b_k} + \omega^{b_{k+1}}) -b^g_k \tag{15} $$

### 状態の誤差
$$ x = \hat{x} + \delta{x} \tag{16} $$
* $\hat{x}$ 真値
* $\delta{x}$ 誤差
* $f$ 状態変化
$$ x_{k+1} + \delta{x}_{k+1} = f(x_{k}, u_{k}) \tag{17} $$
$$ x_{k+1} + \delta{x}_{k+1}  =  f(\hat{x} + \delta{x}, \hat{u} + \delta{n}) \tag{18} $$

$$ \hat{x}_{k+1} + \delta{x}_{k+1} = f(\hat{x}_{k}, \hat{u}_{k}) + F \delta{x_{k}} + G n_{k} \tag{19} $$
* $\hat{x}_{k}$ kの状態の真値
* $\hat{u}_{k}$ kのinput
$$ F = \frac{\partial f(x_{k}, u_{k})}{\partial \delta{x_{k}}} \tag{20} $$

$$ F = \lim_{\delta{x_k} \to 0} \frac{ \partial f(\hat{x} + \delta{x_k}) - f(\hat{x})}{ \delta{x_{k}} } \tag{21} $$

$$ G = \frac{\partial f(x_{k}, u_{k})}{\partial \delta{n_{k}}} $$


### IMU誤差状態
$$ \delta{x}=(\delta{p},\delta{\theta},\delta{v},\delta{b_a},\delta{b_g}) \tag{22} $$



* 位置の誤差: $\delta{p}$
* 姿勢の誤差: $\delta{\theta}$
* 速度の誤差: $\delta{v}$ 
* 加速度biasの誤差: $\delta{b_a}$
* gyro biasの誤差: $\delta{b_g}$



$$ F = \left[
\begin{matrix} 
I & f_{12} &  I \delta t & -\frac{1}{2}(q_{b_{i} b_{k}} + q_{b_{i} b_{k+1}}) \delta t^2 & f_{15}
\\ 
0 & f_{22} & 0 & 0 & I \delta t
\\ 
0 & f_{32} & I & -\frac{1}{2}(q_{b_{i} b_{k}} + q_{b_{i} b_{k+1}}) \delta t  & f_{35}
\\ 
0 & 0 & 0 & I & 0
\\ 
0 & 0 & 0 & 0 & I
 \end{matrix}
\right] \tag{23} $$

### 証明：$f_{12}$

$$ f_{12} = \frac{\partial \delta p_{k+1} }{\partial \delta \theta_k} $$

according to (9): $$ p_{b_{k+1}} = p_{b_{k}} + v_{b_k}\delta{t} + \frac{1}{4} ( R_{b_i,b_{k}}(a^{b_k} - b^a_k) + R_{b_i, b_{k+1}}(a^{b_{k+1}} -b^a_k) ) \delta{t^2}   $$

according to (25)
$$ p_{b_{k+1}}(+noise) = p_{b_{k}} + v_{b_k}\delta{t} + \frac{1}{4} ( R_{b_i,b_{k}}(I + [\delta \theta_k]_\times)(a^{b_k} - b^a_k) + R_{b_i, b_{k+1}} (I + [\delta \theta_k]_\times) (I - [\omega \delta t]_\times)  (a^{b_{k+1}} -b^a_k) ) \delta{t^2}  $$
$$ \frac{\partial \delta \theta_{k+1} }{\partial \delta \theta_k} = \frac{ \frac{1}{4} ( R_{b_i,b_{k}} [\delta \theta]_\times(a^{b_k} - b^a_k) + R_{b_i, b_{k+1}} (I - [\omega \delta t]\times) [\delta \theta_k]_\times (a^{b_{k+1}} -b^a_k) ) \delta{t^2} }{\delta \theta_k} $$
クロスの交代性
$$ \frac{\partial \delta \theta_{k+1} }{\partial \delta \theta_k} = - \frac{1}{4} ( R_{b_i,b_{k}} [a^{b_k} - b^a_k]_\times + R_{b_i, b_{k+1}} (I - [\omega \delta t]_\times)  [a^{b_{k+1}} -b^a_k]_\times ) \delta{t^2}  \tag{24} $$



### 証明：$f_{15}$

$$ f_{15} = \frac{\partial \delta p_{k+1} }{\partial \delta b_g} $$

according to (9): $$ p_{b_{k+1}} = p_{b_{k}} + v_{b_k}\delta{t} + \frac{1}{4} ( R_{b_i,b_{k}}(a^{b_k} - b^a_k) + R_{b_i, b_{k+1}}(a^{b_{k+1}} -b^a_k) ) \delta{t^2}   $$

according to (15)
$$ p_{b_{k+1}}(+noise) = p_{b_{k}} + v_{b_k}\delta{t} + \frac{1}{4} ( R_{b_i,b_{k}}(a^{b_k} - b^a_k) + R_{b_i, b_{k+1}} (I - [b_g \delta t]_\times)  (a^{b_{k+1}} -b^a_k) ) \delta{t^2}  $$
$$ \frac{\partial \delta \theta_{k+1} }{\partial \delta b_g} = \frac{ \frac{1}{4} ( R_{b_i, b_{k+1}} ( - [b_g \delta t]_\times) (a^{b_{k+1}} -b^a_k) ) \delta{t^2} }{\delta b_g} $$
クロスの交代性
$$ f_{12} = \frac{\partial \delta \theta_{k+1} }{\partial \delta \theta_k} = - \frac{1}{4} (R_{b_i, b_{k+1}}  [a^{b_{k+1}} -b^a_k]_\times ) \delta{t^2} (-\delta{t})  \tag{25} $$


### 証明：$f_{22}$

$$ f_{22} = \frac{\partial \delta \theta_{k+1}}{\partial \delta \theta_{k}} $$
$$q_{b_i b_{k+1}} \otimes \left[
\begin{matrix} 1 \\ \delta \theta_{k+1} /2 \end{matrix}
\right] 
= q_{b_i b_{k}} \otimes \left[
\begin{matrix} 1 \\ \delta \theta_{k} /2 \end{matrix}
\right]  \otimes \left[
\begin{matrix} 1 \\ \omega \delta t /2 \end{matrix}
\right] $$

$$ \left[
\begin{matrix} 1 \\ \delta \theta_{k+1} /2 \end{matrix}
\right] 
= q_{b_{k+1} b_{i}} q_{b_i b_{k}} \otimes \left[
\begin{matrix} 1 \\ \delta \theta_{k} /2 \end{matrix}
\right]  \otimes \left[
\begin{matrix} 1 \\ \omega \delta t /2 \end{matrix}
\right] $$

$$ \left[
\begin{matrix} 1 \\ \delta \theta_{k+1} /2 \end{matrix}
\right] 
= q_{b_{k+1} b_{k}} \otimes \left[
\begin{matrix} 1 \\ \delta \theta_{k} /2 \end{matrix}
\right]  \otimes \left[
\begin{matrix} 1 \\ \omega \delta t /2 \end{matrix}
\right] $$

$$ \left[
\begin{matrix} 1 \\ \delta \theta_{k+1} /2 \end{matrix}
\right] 
= \left[
\begin{matrix} 1 \\ -\omega \delta t /2 \end{matrix}
\right]  \otimes \left[
\begin{matrix} 1 \\ \delta \theta_{k} /2 \end{matrix}
\right]  \otimes \left[
\begin{matrix} 1 \\ \omega \delta t /2 \end{matrix}
\right] $$

$$ 
= \left[
\begin{matrix} 1 \\ R \delta \theta_{b_k} /2 \end{matrix}
\right]$$

$$ \delta \theta_{k+1} /2 = R \delta \theta_{b_k} /2 $$
$$ \delta \theta_{k+1} = (I - [\omega \delta t]_\times) \delta \theta_{b_k}  $$
$$ f_{22} = I - [\omega \delta t]_\times  \tag{26} $$

### 証明：$f_{32}$

$$ f_{32} = \frac{\partial \delta v_{k+1} }{\partial \delta \theta_k} $$

$$ v_{b_{k+1}} = v_{b_{k}} + \frac{1}{2}[ R_{b_i,b_{k}}(a^{b_k} - b^a_k) - g^w + R_{b_i, b_{k+1}}(a^{b_{k+1}} -b^a_k)-g^w ]\delta{t} $$


$$ v_{b_{k+1}}(+noise) = v_{b_{k}} + \frac{1}{2}[ R_{b_i,b_{k}}(I + [\delta \theta_k]_\times)(a^{b_k} - b^a_k) + R_{b_i, b_{k+1}} (I + [\delta \theta_k]_\times) (I - [\omega \delta t]_\times)  (a^{b_{k+1}} -b^a_k)-g^w ]\delta{t} $$

$$ \frac{\partial \delta v_{k+1} }{\partial \delta \theta_k} = \frac{ \frac{1}{2} ( R_{b_i,b_{k}} [\delta \theta]_\times(a^{b_k} - b^a_k) + R_{b_i, b_{k+1}} (I - [\omega \delta t]_\times) [\delta \theta_k]_\times (a^{b_{k+1}} -b^a_k) ) \delta{t} }{\delta \theta_k} $$
クロスの交代性
$$ \frac{\partial \delta \theta_{k+1} }{\partial \delta \theta_k} = - \frac{1}{2} ( R_{b_i,b_{k}} [a^{b_k} - b^a_k]_\times + R_{b_i, b_{k+1}} (I - [\omega \delta t]_\times)  [a^{b_{k+1}} -b^a_k]_\times ) \delta{t}  \tag{27} $$


### 証明：$f_{35}$

$$ f_{35} = \frac{\partial \delta v_{k+1} }{\partial \delta b_g} $$

according to (11): $$ v_{b_{k+1}} = v_{b_{k}} +  \frac{1}{2} ( R_{b_i,b_{k}}(a^{b_k} - b^a_k) + R_{b_i, b_{k+1}}(a^{b_{k+1}} -b^a_k) ) \delta{t}   $$

according to (15)
$$ v_{b_{k+1}}(+noise) = v_{b_{k}} + \frac{1}{2} ( R_{b_i,b_{k}}(a^{b_k} - b^a_k) + R_{b_i, b_{k+1}} (I - [b_g \delta t]_\times)  (a^{b_{k+1}} -b^a_k) ) \delta{t}  $$
$$ \frac{\partial \delta \theta_{k+1} }{\partial \delta b_g} = \frac{ \frac{1}{2} ( R_{b_i, b_{k+1}} ( - [b_g \delta t]_\times) (a^{b_{k+1}} -b^a_k) ) \delta{t} }{\delta b_g} $$
クロスの交代性
$$ f_{35} = \frac{\partial \delta \theta_{k+1} }{\partial \delta \theta_k} = - \frac{1}{2} (R_{b_i, b_{k+1}}  [a^{b_{k+1}} -b^a_k]_\times ) \delta{t} (-\delta{t})  \tag{28} $$


ノイズの分析
$$ [n^a_k,n^g_k,n^a_{k+1},n^g_{k+1},n_{ba_k},n_{bg_k}] $$

$$ G = \left[
\begin{matrix} 
\frac{1}{4} R_{b_i,b_k} \delta{t^2} & g_{12} &  \frac{1}{4} R_{b_i,b_{k+1}} \delta{t^2} & g_{14} & 0 & 0
\\ 
0 & \frac{1}{2} I \delta{t} & 0 & \frac{1}{2} I \delta{t} & 0 & 0
\\ 
\frac{1}{2} R_{b_i,b_k} \delta{t} & g_{32} &  \frac{1}{2} R_{b_i,b_{k+1}} \delta{t} & g_{34} & 0 & 0
\\ 
0 & 0 & 0 & 0 & I \delta{t} & 0
\\ 
0 & 0 & 0 & 0 & 0 & I \delta{t}
 \end{matrix}
\right] \tag{29} $$

### 証明：$g_{11}$
$$ g_{11} = \frac{\partial  p_{k+1} }{\partial  n^a_k} $$

ノイズを考慮すると(14)(15)は以下:
 $$ a = \frac{1}{2} [ R_{wb_{k}}(a^{b_k} + n^a_k - b^a_k) + R_{w b_{k+1}}(a^{b_{k+1}} + n^a_{k+1} -b^a_k) ] - g^w \tag{30} $$
 $$ \omega = \frac{1}{2} (\omega^{b_k} + n^g_k + \omega^{b_{k+1}} + n^g_{k+1} ) -b^g_k \tag{31} $$

according to (9): $$ p_{b_{k+1}} = p_{b_{k}} + v_{b_k}\delta{t} + \frac{1}{2} a \delta{t^2}   $$
$$ \frac{\partial  p_{k+1} }{\partial  n^a_k}= \frac{\partial  a }{\partial  n^a_k} = \frac{1}{4} R_{b_i,b_k} \delta{t^2} \tag{32}$$


### 証明：$g_{12}$, $g_{14}$
$$ g_{12} = \frac{\partial  p_{k+1} }{\partial  n^g_k} $$
$$ g_{14} = \frac{\partial  p_{k+1} }{\partial  n^g_{k+1}} $$


$$ p_{b_{k+1}} = p_{b_{k}} + v_{b_k}\delta{t} + \frac{1}{2} a \delta{t^2}   $$ 
a <- (30) 
$$ \frac{\partial  p_{k+1} }{\partial  n^g_k} = \frac{1}{4} R_{b_i,b_{k+1}} [\frac{1}{2} n^g_k \delta{t}]_\times (a^{b_{k+1}}  -b^a_k)\delta{t^2} / (\partial  n^g_k)  $$
$$ = -\frac{1}{4} R_{b_i,b_{k+1}} [a^{b_{k+1}}  -b^a_k]_\times \delta{t^2} (\frac{1}{2} \delta{t}) \tag{33} $$
$$ \frac{\partial  p_{k+1} }{\partial  n^g_{k+1}} = -\frac{1}{4} R_{b_i,b_{k+1}} [a^{b_{k+1}}  -b^a_k]_\times \delta{t^2} (\frac{1}{2} \delta{t}) \tag{34} $$


### 証明：$g_{32}$, $g_{34}$
$$ g_{12} = \frac{\partial  v_{k+1} }{\partial  n^g_k} $$
$$ g_{14} = \frac{\partial  v_{k+1} }{\partial  n^g_{k+1}} $$


$$ v_{b_{k+1}} = v_{b_{k}} + a_{b_k}\delta{t}   $$
$$ \frac{\partial  v_{k+1} }{\partial  n^g_k}= \frac{\partial  a }{\partial  n^g_k} = \frac{1}{2} R_{b_i,b_{k+1}} [\frac{1}{2} n^g_k \delta{t}]_\times (a^{b_{k+1}}  -b^a_k)\delta{t} / (\partial  n^g_k)  $$
$$ = -\frac{1}{2} R_{b_i,b_{k+1}} [a^{b_{k+1}}  -b^a_k]_\times \delta{t} (\frac{1}{2} \delta{t}) \tag{35} $$
$$ \frac{\partial  p_{k+1} }{\partial  n^g_{k+1}} = -\frac{1}{2} R_{b_i,b_{k+1}} [a^{b_{k+1}}  -b^a_k]_\times \delta{t} (\frac{1}{2} \delta{t}) \tag{36} $$
